In [1]:
from coffea.nanoevents import NanoEventsFactory
from coffea.nanoevents.schemas import BaseSchema
from config.selectionconfig import runsetting as rs

In [2]:
from analysis.selutility import *

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from config.selectionconfig import settings as sel_cfg

In [5]:
events = NanoEventsFactory.from_root(
        f"{rs.SINGLE_PATH}:Events",
        entry_stop=None,
        metadata={"dataset": "DYJets"},
        uproot_options={"timeout": 100},
        schemaclass=BaseSchema,
    ).events()

In [10]:
sels = EventSelections()

In [11]:
channel_cfg = sel_cfg.signal.channel1.selections

In [14]:
sels.lepselcfg = sel_cfg.signal.channel1.selections
sels.lepselsetter(events)

In [29]:
cutflow = sels.leptonsel.cutflow(*sels.leptonsel.names)

In [40]:
cutflow.print()

/Users/yuntongzhou/miniconda3/envs/dev/lib/python3.9/site-packages/coffea/analysis_tools.py:845: UserWarning: Printing the cutflow statistics is going to compute dask_awkward objects.
  warnings.warn(


Cutflow stats:
Cut ElectronSelection   :pass = 11703               cumulative pass = 11703               all = 17901               -- eff = 65.4 %                    -- cumulative eff = 65.4 %
Cut MuonSelection       :pass = 2318                cumulative pass = 1746                all = 17901               -- eff = 12.9 %                    -- cumulative eff = 9.8 %
Cut TauSelection        :pass = 9305                cumulative pass = 1020                all = 17901               -- eff = 52.0 %                    -- cumulative eff = 5.7 %


In [47]:
res = cutflow.result().nevcutflow

In [48]:
res

[17901, 11703, 1746, 1020]

In [72]:
sels.lepselcfg

<Box: {'electron': {'veto': True, 'ptLevel': 25}, 'muon': {'veto': False, 'pt': 25, 'eta': 2.1, 'iso': 0.15, 'count': 1, 'dxy': 0.045, 'dz': 0.2}, 'tau': {'veto': False, 'pt': 35, 'eta': 2.3, 'IDvsjetLevel': 5, 'IDvsmuLevel': 4, 'IDvseLevel': 3, 'dz': 0.2, 'count': 1}, 'pair': {'name': 'MT', 'OS': True, 'dRLevel': 0.5}}>

In [11]:
def measure_execution_time(func, *args, **kwargs):
    start_time = time.perf_counter()  # Start timing

    # Call the function with any provided arguments
    result = func(*args, **kwargs)

    end_time = time.perf_counter()  # End timing
    execution_time = end_time - start_time

    print(f"The function '{func.__name__}' took {execution_time} seconds to execute.")
    return result

In [36]:
start_time = time.perf_counter()
events = NanoEventsFactory.from_root(
        f"{rs.SINGLE_PATH}:Events",
        entry_stop=None,
        metadata={"dataset": "DYJets"},
        uproot_options={"timeout": 100},
        schemaclass=BaseSchema,
    ).events()
end_time = time.perf_counter()
execution_time = end_time - start_time
print(f"The function took {execution_time} seconds to execute.")

The function took 0.6647109169998657 seconds to execute.


In [59]:
jetpt = events["Jet_pt"].compute()
jeteta = events["Jet_eta"].compute()
dak_zipped = dak.zip({"pt": dak.from_awkward(jetpt, npartitions=10),
                      "eta": dak.from_awkward(jeteta, npartitions=10)})

In [16]:
start_time = time.perf_counter()  # Start timing
(dak.num(dak_zipped)==2).compute()
end_time = time.perf_counter()
execution_time = end_time - start_time
print(f"The function took {execution_time} seconds to execute.")

The function took 0.02522283299981609 seconds to execute.


In [64]:
dak_zipped = dak_zipped[dak_zipped.pt>50]

In [65]:
dak_mask = dak.num(dak_zipped)>2

In [66]:
dak_zipped[dak_mask].compute()

<Array [[{pt: 195, eta: ..., ...}, ...], ...] type='8036 * var * {pt: float...'>

In [31]:
# correct way to compute events
events=events[dak_mask.compute()]

In [90]:
start_time = time.perf_counter()  # Start timing
(dak.num(events["Jet_pt"])==2).compute()
end_time = time.perf_counter()
execution_time = end_time - start_time
print(f"The function took {execution_time} seconds to execute.")

The function took 0.894880166000803 seconds to execute.


In [89]:
import operator as op
op.eq(dak.num(dak_zipped), 2)

dask.awkward<equal, npartitions=10>

In [2]:
lepsel = PackedSelection()

In [3]:
type(lepsel)

coffea.analysis_tools.PackedSelection